In [ ]:
# imported from Pranav Sanghavi's notebook 
import glob
import os
import time

import h5py
import matplotlib.pylab as plt
import numpy as np
import numpy.ma as ma
from matplotlib.offsetbox import AnchoredText
from matplotlib.pylab import colorbar 
from skrf import Network, Frequency
import math
import psutil
import matplotlib.cm as cm

style = 'fast'
tick_dir = 'in'
plt.style.use(style)
plt.rcParams['xtick.direction'] = tick_dir
plt.rcParams['ytick.direction'] = tick_dir
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['font.size'] = 15
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.5
plt.rcParams.update({
    "text.usetex": False,
    "font.family": "serif"})

###################################################################################
# Analog chain labels
###################################################################################

#pathtoacfile = "/home/observer/tone_jupyter/anachainz.csv"
#ac = np.genfromtxt(pathtoacfile, delimiter=",", names=True,
#                   dtype=(int, int, 'U5', 'U5', int, 'U2', int))
#inputs = ac['feed']

def get_beam_fwhm(d=6.0, nu=600):
    fwhm = constants.c/(nu*1e6)/d   # lambda/D
    return np.rad2deg(fwhm * 1.22)

def gaussian(x, a, mean, sigma):
    val = a * np.exp(-(x - mean)**2 / sigma**2)
    return val

def scale_zero_one(x):
    return (x - x.min())/(x.max() - x.min())

def seconds_to_degrees(seconds, declination=22.0145):
    """
    convert seconds to degrees of sky motion at a latitude default TONE latitude default for cra
    """
    return seconds * np.cos(np.deg2rad(declination)) / 240

def read_corr_mode_h5(file_path, f_start=None, f_stop=None):
    """
    Reading all hdf5 files
    """
    class correlator_data:
        def __init__(self, vis, time, sat, index_map):
            self.vis = vis
            self.time = time
            self.sat = sat
            self.freq = index_map["freq"][:]
            self.prod = index_map['prod'][:]
    files = glob.glob(file_path + "*[!.lock]")
    files.sort()
    
    #grab different sets of files.  
    if f_start is not None:
        if f_stop is not None:
            files = files[f_start:f_stop]
        else:
            files = files[f_start:]
    if f_start is None:
        if f_stop is not None:
            files = files[:f_stop]

    try:
        t = 0
        for x in files:
            print(f"Reading File: {x}")
            f = h5py.File(x, "r")
            index_map = f['index_map']
            if(t == 0):
                vis = f['vis'][:]
                time = index_map['time'][:]
                sat = f['sat'][:]
                print("Appended vis data and time stream from %s" % (x))
                t = t + 1
            else:
                time = np.append(time, index_map['time'][:])
                vis = np.append(vis, f['vis'][:], axis=0)
                sat = np.append(sat, f['sat'][:], axis=0)
                print("Appended vis data and time stream from %s" % (x))
        
        return correlator_data(
            vis,
            time,
            sat,
            index_map
        )
    finally:
        print("Done!!!!!!!")

def get_mag_phase(power0):
    """
    # return magnitude and phase in degrees of correlations
    """
    return np.absolute(power0), np.rad2deg(np.angle(power0))

def plot_waterfall_corr_data(data, corr_indices):
    plt.figure(figsize=(20, 20))
    dim1 = math.floor(math.sqrt(len(corr_indices)))
    dim2 = math.ceil(len(corr_indices)/dim1)
    for i,auto in enumerate(corr_indices):
        plt.subplot(dim1, dim2, i + 1)
        wfall = data[:, :, auto] # auto -> corr index 
        plt.imshow(wfall, vmin = np.nanpercentile(wfall,5), vmax =np.nanpercentile(wfall,95), aspect='auto')
        plt.xlabel("freq index")
        plt.ylabel("time index")
        plt.title(f"corr {corr_data.prod[auto]}")
    plt.tight_layout()
    
def plot_freq_channel_corr_data(pow_mag, freq_index, freq, corr_indices):
    plt.figure(figsize=(20, 20))
    dim1 = math.floor(math.sqrt(len(corr_indices)))
    dim2 = math.ceil(len(corr_indices)/dim1)
    for index, corr_index in enumerate(corr_indices):
            plt.subplot(dim1, dim2, index + 1)
            plt.title(f"Corr: {corr_data.prod[corr_index]}")
            #plt.plot(pow_mag[:, freq_index, corr_index]00000053.h5)
            plt.plot(pow_mag[:, :, corr_index])

    plt.suptitle(f"Frequency {freq} MHz")
    plt.tight_layout()


In [ ]:
f = h5py.File("/Volumes/EXTRSSD/digital_gains_corr/20240724T205100Z_WVUc52_digitalgain/00000056.h5","r")

In [ ]:
# plotting digital gains --

print(list(f.attrs))
print(f.attrs['acquisition_name'])
print(f.attrs['archive_version'])
print(f.attrs['collection_server'])
print(f.attrs['git_version_tag'])
print(f.attrs['instrument_name'])
print(f.attrs['notes'])
print(f.attrs['system_user'])
print(f.attrs['type'])
print(f.attrs['version'])

print(list(f.items()))
print((f['compute_time']))
print((f['index_map']))
print((f['gain_coeff']))
print((f['gain_exp']))

inputs = (f['index_map']['input'][:])
frqs = f['index_map']['freq'][:]
gaincoeff = f['gain_coeff'][()]
gainexp = f['gain_exp'][()]
gain = gaincoeff.real*np.exp2(gainexp)

plt.figure(figsize=[16,18])
print(f"Calculating digital gains of the channels...")
i = 0
for i in np.arange(0,2,1):  
    plt.subplot(8,2,i+1)
    #Digital gain of every channel
    plt.scatter(frqs['centre'], gain[0][:,i]) # i referes to input channel (total 16 ch on iceboard); 
    #plt.title( " %s and gain_exp = %s and rms %s for rms i/p %s" %(inputs[i], gainexp[:,i], rms[i][2], rms[i][0][2]))
    plt.title( " %s " %(inputs[i]))
plt.tight_layout()

In [ ]:
# gains related stats -- 

print(gainexp)
print(gaincoeff[0,:,1])
gain = gaincoeff.real*np.exp2(gainexp)
print(gain.shape)
print(gain[0,:,1])
print(gain[0,:,1].shape)
print(frqs['centre'])
print(inputs)

In [ ]:
# reading correlation files 
file_path = '/Volumes/EXTRSSD/corr_dat4/20240724T205100Z_WVUc52/corr/*/*' 
corr_data = read_corr_mode_h5(file_path, f_start=56, f_stop=88)
print("getting autocorrelation magnitudes and phase")
pow_mag, pow_phase = get_mag_phase(corr_data.vis)
print("done getting pow_mag and pow_phase")
sat_data = corr_data.sat.real # what is this paramater?

In [ ]:
# visibility data 
corr_data.vis[1][:][31] # correlation data having real and imag component 
print(corr_data.vis[:,:,:].shape) # time bins - freq bins - corr indices

In [ ]:
# plotting correlation powers against time index for specific frequency bina 

colorsarr=cm.gnuplot2(np.linspace(0,1,1024))
plt.figure(figsize=[15,10])
#128 mhz ranges from freq bins 574 to 901 (576MHz to 448MHz)
fbin= 600
k=50
tmin,tmax=[6903,8106]

plt.subplot(3,1,1)
plt.plot(pow_mag[tmin:tmax,fbin,0], c=colorsarr[fbin])
#plt.ylim(0,80000)
plt.title(f"Corr: (0,0)")
plt.xlabel("time index")
plt.ylabel("corr power (arb)")

plt.subplot(3,1,2)
plt.plot(pow_mag[tmin:tmax,fbin,1], c=colorsarr[fbin]) 
#plt.ylim(0,80000)
plt.title(f"Corr: (0,1)")
plt.xlabel("time index")
plt.ylabel("corr power (arb)")

plt.subplot(3,1,3)
plt.plot(pow_mag[tmin:tmax,fbin,4],c=colorsarr[fbin]) 
#plt.ylim(0,80000)
plt.title(f"Corr: (1,1)")
plt.xlabel("time index")
plt.ylabel("corr power (arb)")

plt.suptitle(f"Pow_magnitude")
plt.tight_layout()

delta_time = np.diff(corr_data.time["irigb_time"])[1]*1e-9
print(f"Each time bin is: {delta_time} seconds")
print(f"total time bins (considered for the plots) are {len(corr_data.vis[tmin:tmax,0,0])}")
print(f"lowest time bin {tmin} and highest time bin {tmax}")
print(f"{delta_time*len(corr_data.vis[tmin:tmax,0,0])} seconds of data (considered for the plots)")

In [ ]:
# 1 sec contains 23.4818579 time bins ~24 time bins
# 0.5 sec -- 11.920929 ~12 time bins
# calculating medians for the set of x pulses  

med0, med1, med4 = [], [], []
step = 11 # 11 time bins refer to ON points in one pulse and 24 is roughly one complete pulse 
for i in np.arange(6903,7623,24): # need to give range manually every time depending on the sections of the dataset
    med0.append(np.median(pow_mag[i:i+step,fbin,0])) 
    med1.append(np.median(pow_mag[i:i+step,fbin,1])) 
    med4.append(np.median(pow_mag[i:i+step,fbin,4])) 

In [ ]:
# plotting standard deviations of medians (for few pulses) to see deviations in each channel 

fig = plt.figure(figsize=[8,5])
ax = plt.subplot(111)

ax.plot(med0, marker='o', label='corr: (0,0)')
ax.plot(med1, marker='*', label='corr: (0,1)')
ax.plot(med4, marker='.', label='corr: (1,1)')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('pulses')
plt.ylabel('median (arb units)')

print(f"corr(0,0) : standard deviation of medians of corr powers for 30 sec pulses is {np.std(med0)}")
print(f"corr(0,1) : standard deviation of medians of corr powers for 30 sec pulses is {np.std(med1)}")
print(f"corr(1,1) : standard deviation of medians of corr powers for 30 sec pulses is {np.std(med4)}")

In [ ]:
# standard deviation in single ON pulse 

print(np.std(pow_mag[6903:6914,fbin,0]))
print(np.std(pow_mag[6903:6914,fbin,1]))
print(np.std(pow_mag[6903:6914,fbin,4]))

In [ ]:
# overplotting autos and crosses power to check difference

fig = plt.figure(figsize=[15,7])
ax = plt.subplot(111)

ax.plot(pow_mag[tmin:tmax,fbin,0], c=colorsarr[k],label='Corr: (0,0)' )
ax.plot(pow_mag[tmin:tmax,fbin,1],c=colorsarr[k+300], label='Corr: (0,1)')
ax.plot(pow_mag[tmin:tmax,fbin,4],c=colorsarr[k+600], label='Corr: (1,1)')
plt.xlabel("time index")
plt.ylabel("corr power (arb)")
plt.title(f"Corr power mag")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

In [ ]:
# plotting single 1 sec time samples across the 400-800MHz band 

colorsarr=cm.gnuplot2(np.linspace(0,1,1024))
plt.figure(figsize=[15,10])
k=100
tb_1=6909
tb_2=7500
tb_3=8097
plt.subplot(3,1,1)
plt.plot(pow_mag[tb_1,:,1], c=colorsarr[k]) 
#plt.ylim(20000,50000)
plt.title("time bin {} -- 0x1".format(tb_1))

plt.subplot(3,1,2)
plt.plot(pow_mag[tb_2,:,1], c=colorsarr[k+200]) 
#plt.ylim(20000,60000)
plt.title("time bin {}-- 0x1".format(tb_2))

plt.subplot(3,1,3)
plt.plot(pow_mag[tb_3,:,1], c=colorsarr[k+400])
#plt.ylim(0,150000)
plt.title("time bin {} -- 0x1".format(tb_3))

plt.suptitle(f"freq_dependence of single time bin")
plt.tight_layout()

In [ ]:
# plotting waterfall --

plot_waterfall_corr_data(pow_mag[:,:,:],[0,1,4])

In [ ]:
# difference in corr powers between channels (0,0) and (0,1)

plt.plot(pow_mag.real[:,fbin,0] - pow_mag.real[:,fbin,1], c="magenta")

In [ ]:
## plot time bin durations for whole data (to see any glitch)

plt.figure()
plt.plot(np.diff(corr_data.time["irigb_time"])[1:], c="blue")
delta_time = np.diff(corr_data.time["irigb_time"])[1]*1e-9
print(f"Each time bin is: {delta_time} seconds")

In [ ]:
# Phase variation for a single time bin as a function of frequency -- function defined 

def plot_freq_channel_corr_data(pow_mag, freq_index, freq, corr_indices):
    fig= plt.figure(figsize=(10, 5))
    dim1 = math.floor(math.sqrt(len(corr_indices)))
    dim2 = math.ceil(len(corr_indices)/dim1)
    fbin = 3570
    for index, corr_index in enumerate(corr_indices):
            ax = plt.subplot(dim1, dim2, index + 1)
            plt.title(f"Corr: {corr_data.prod[corr_index]}")
            ax.plot(pow_mag[8097,500:950, corr_index], c=colorsarr[500], label='tbin 8097')
            #ax.plot(pow_mag[3710,510:550, corr_index], c=colorsarr[700], label='tbin 3710')
            #ax.plot(pow_mag[3860,510:550, corr_index], c=colorsarr[900], label='tbin 3860')
            #ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
            ax.legend()
            plt.xlabel(f"frequency index")
            plt.ylabel(f"phase (degrees)")

    plt.suptitle(f"Correlated noise phase - frequency: 500-950MHz")
    plt.tight_layout()

In [ ]:
# Phase variation for a single time bin as a function of frequency -- calling a function

print(corr_data.freq) # from corr file 
#print(f['index_map']['freq'][:]) # from digital gain file 
frequency_index = 200  # why 200? -> 200th freq bin refers to 721.875 MHz
frequency = corr_data.freq[0][0] # freq at that frequency bin 
plot_freq_channel_corr_data(pow_phase,frequency_index,frequency,[1])

In [ ]:
##-- write code again for determining the slopes | FIX THIS!!! --##

dx1=np.diff(np.arange(0,1024,1))
dy1=np.diff(pow_phase[2947:2957,fbin,1])
slopes1 = dy1/dx1

dx2=np.diff(np.arange(0,1024,1))
dy2=np.diff(pow_phase[2947:2957,fbin,1])
slopes2=dy2/dx2

dx3=np.diff(np.arange(0,1024,1))
dy3=np.diff(pow_phase[2947:2957,fbin,1])
slopes3=dy3/dx3

# determining std of those slopes 
std1=np.std(slopes1)
std2=np.std(slopes2)
std3=np.std(slopes3)
fig = plt.figure()
ax = plt.subplot(111)

ax.plot(std1, c=colorsarr[100], label='slopes1')
ax.plot(std2, c=colorsarr[500], label='slopes2')
ax.plot(std3, c=colorsarr[800], label='slopes3')
ax.legend()

plt.show()

In [ ]:
## plotting phase curve slopes for different correlation powers 

fig = plt.figure()
ax = plt.subplot(111)

ax.plot(slopes1, c=colorsarr[100],label='time bin - 3570')
ax.plot(slopes2, c=colorsarr[500],label='time bin - 3710')
ax.plot(slopes3, c=colorsarr[800],label='time bin - 3860')

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

In [ ]:
# Phase variation for a single frequency bin as a function of time -- function defined 

def plot_freq_channel_corr_data(pow_phase, freq_index, corr_indices):
    plt.figure(figsize=(25, 10))
    dim1 = math.floor(math.sqrt(len(corr_indices)))
    dim2 = math.ceil(len(corr_indices)/dim1)
    for index, corr_index in enumerate(corr_indices):
            plt.subplot(dim1, dim2, index + 1)
            plt.title(f"Corr: {corr_data.prod[corr_index]}")
            plt.plot(pow_phase[4000:4500,freq_index, corr_index])

    #plt.suptitle(f"Frequency {freq} MHz")
    plt.tight_layout()

In [ ]:
# Phase variation for a single frequency bin as a function of time -- calling a function

frequency_index = np.arange(600,850,1)
frequency = corr_data.freq[frequency_index][0]
print(f"Frequency: {frequency}")
plot_freq_channel_corr_data(pow_phase,frequency_index,[1])

In [ ]:
#### beam pattern ###

In [ ]:
## -- correlator data 
# reading the required only corr files for plotting beam pattern
file_path = '/Volumes/EXTRSSD/corr_dat4/20240724T205100Z_WVUc52/corr/*/*' 

# reading files for 0 degrees angle between rx and tx
corr_data0 = read_corr_mode_h5(file_path, f_start=40, f_stop=42)
pow_mag0, pow_phase0 = get_mag_phase(corr_data0.vis)
# reading files for negative 10 degrees angle between rx and tx
corr_data10n = read_corr_mode_h5(file_path, f_start=57, f_stop=59)
pow_mag10n, pow_phase10n = get_mag_phase(corr_data10n.vis)
# reading files for negative 20 degrees angle between rx and tx
corr_data20n = read_corr_mode_h5(file_path, f_start=63, f_stop=65)
pow_mag20n, pow_phase20n = get_mag_phase(corr_data20n.vis)
# reading files for negative 30 degrees angle between rx and tx
corr_data30n = read_corr_mode_h5(file_path, f_start=70, f_stop=72)
pow_mag30n, pow_phase30n = get_mag_phase(corr_data30n.vis)
# reading files for negative 40 degrees angle between rx and tx
corr_data40n = read_corr_mode_h5(file_path, f_start=78, f_stop=80)
pow_mag40n, pow_phase40n = get_mag_phase(corr_data40n.vis)
# reading files for negative 50 degrees angle between rx and tx
corr_data50n = read_corr_mode_h5(file_path, f_start=83, f_stop=85)
pow_mag50n, pow_phase50n = get_mag_phase(corr_data50n.vis)
# reading files for negative 60 degrees angle between rx and tx
corr_data60n = read_corr_mode_h5(file_path, f_start=90, f_stop=92)
pow_mag60n, pow_phase60n = get_mag_phase(corr_data60n.vis)
# reading files for negative 70 degrees angle between rx and tx
corr_data70n = read_corr_mode_h5(file_path, f_start=98, f_stop=100)
pow_mag70n, pow_phase70n = get_mag_phase(corr_data70n.vis)
# reading files for negative 80 degrees angle between rx and tx 
corr_data80n = read_corr_mode_h5(file_path, f_start=104, f_stop=106)
pow_mag80n, pow_phase80n = get_mag_phase(corr_data80n.vis)
# reading files for negative 90 degrees angle between rx and tx 
corr_data90n = read_corr_mode_h5(file_path, f_start=111, f_stop=113)
pow_mag90n, pow_phase90n = get_mag_phase(corr_data90n.vis)
# reading files for negative 100 degrees angle between rx and tx 
corr_data100n = read_corr_mode_h5(file_path, f_start=118, f_stop=120)
pow_mag100n, pow_phase100n = get_mag_phase(corr_data100n.vis)
# reading files for negative 110 degrees angle between rx and tx 
corr_data110n = read_corr_mode_h5(file_path, f_start=124, f_stop=126)
pow_mag110n, pow_phase110n = get_mag_phase(corr_data110n.vis)
# reading files for negative 120 degrees angle between rx and tx 
corr_data120n = read_corr_mode_h5(file_path, f_start=132, f_stop=134)
pow_mag120n, pow_phase120n = get_mag_phase(corr_data120n.vis)
# reading files for negative 130 degrees angle between rx and tx 
corr_data130n = read_corr_mode_h5(file_path, f_start=138, f_stop=140)
pow_mag130n, pow_phase130n = get_mag_phase(corr_data130n.vis)
# reading files for negative 140 degrees angle between rx and tx 
#corr_data140n = read_corr_mode_h5(file_path, f_start=147, f_stop=149)
#pow_mag140n, pow_phase140n = get_mag_phase(corr_data140n.vis)


# reading files for positive 10 degrees angle between rx and tx
corr_data10p = read_corr_mode_h5(file_path, f_start=158, f_stop=160)
pow_mag10p, pow_phase10p = get_mag_phase(corr_data10p.vis)
# reading files for positive 20 degrees angle between rx and tx 
corr_data20p = read_corr_mode_h5(file_path, f_start=164, f_stop=166)
pow_mag20p, pow_phase20p = get_mag_phase(corr_data20p.vis)
# reading files for positive 30 degrees angle between rx and tx
corr_data30p = read_corr_mode_h5(file_path, f_start=170, f_stop=172)
pow_mag30p, pow_phase30p = get_mag_phase(corr_data30p.vis)
# reading files for positive 40 degrees angle between rx and tx 
corr_data40p = read_corr_mode_h5(file_path, f_start=178, f_stop=180)
pow_mag40p, pow_phase40p = get_mag_phase(corr_data40p.vis)
# reading files for positive 50 degrees angle between rx and tx
corr_data50p = read_corr_mode_h5(file_path, f_start=184, f_stop=186)
pow_mag50p, pow_phase50p = get_mag_phase(corr_data50p.vis)
# reading files for positive 60 degrees angle between rx and tx 
corr_data60p = read_corr_mode_h5(file_path, f_start=193, f_stop=195)
pow_mag60p, pow_phase60p = get_mag_phase(corr_data60p.vis)
# reading files for positive 70 degrees angle between rx and tx
corr_data70p = read_corr_mode_h5(file_path, f_start=201, f_stop=203)
pow_mag70p, pow_phase70p = get_mag_phase(corr_data70p.vis)
# reading files for positive 80 degrees angle between rx and tx 
corr_data80p = read_corr_mode_h5(file_path, f_start=207, f_stop=209)
pow_mag80p, pow_phase80p = get_mag_phase(corr_data80p.vis)
# reading files for positive 90 degrees angle between rx and tx 
corr_data90p = read_corr_mode_h5(file_path, f_start=215, f_stop=217)
pow_mag90p, pow_phase90p = get_mag_phase(corr_data90p.vis)
# reading files for positive 100 degrees angle between rx and tx 
corr_data100p = read_corr_mode_h5(file_path, f_start=222, f_stop=224)
pow_mag100p, pow_phase100p = get_mag_phase(corr_data100p.vis)
# reading files for positive 110 degrees angle between rx and tx 
corr_data110p = read_corr_mode_h5(file_path, f_start=229, f_stop=231)
pow_mag110p, pow_phase110p = get_mag_phase(corr_data110p.vis)
# reading files for positive 120 degrees angle between rx and tx 
corr_data120p = read_corr_mode_h5(file_path, f_start=237, f_stop=239)
pow_mag120p, pow_phase120p = get_mag_phase(corr_data120p.vis)
# reading files for positive 130 degrees angle between rx and tx 
corr_data130p = read_corr_mode_h5(file_path, f_start=243, f_stop=245)
pow_mag130p, pow_phase130p = get_mag_phase(corr_data130p.vis)
# reading files for positive 140 degrees angle between rx and tx 
#corr_data140p = read_corr_mode_h5(file_path, f_start=243, f_stop=245)
#pow_mag140p, pow_phase140p = get_mag_phase(corr_data140p.vis)


print("getting autocorrelation magnitudes and phase")
print("done getting pow_mag and pow_phase")

In [ ]:
## averaging 0.5 sec ON pulse datapoints -- correlator data 

fbin = 890
m00=np.mean(pow_mag0[223:234,fbin,0])
m01=np.mean(pow_mag0[223:234,fbin,1])
m04=np.mean(pow_mag0[223:234,fbin,4])


m10n0=np.mean(pow_mag10n[210:221,fbin,0])
m10n1=np.mean(pow_mag10n[210:221,fbin,1])
m10n4=np.mean(pow_mag10n[210:221,fbin,4])
m20n0=np.mean(pow_mag20n[224:235,fbin,0])
m20n1=np.mean(pow_mag20n[224:235,fbin,1])
m20n4=np.mean(pow_mag20n[224:235,fbin,4])
m30n0=np.mean(pow_mag30n[220:231,fbin,0])
m30n1=np.mean(pow_mag30n[220:231,fbin,1])
m30n4=np.mean(pow_mag30n[220:231,fbin,4])
m40n0=np.mean(pow_mag40n[222:233,fbin,0])
m40n1=np.mean(pow_mag40n[222:233,fbin,1])
m40n4=np.mean(pow_mag40n[222:233,fbin,4])
m50n0=np.mean(pow_mag50n[206:217,fbin,0])
m50n1=np.mean(pow_mag50n[206:217,fbin,1])
m50n4=np.mean(pow_mag50n[206:217,fbin,4])
m60n0=np.mean(pow_mag60n[226:237,fbin,0])
m60n1=np.mean(pow_mag60n[226:237,fbin,1])
m60n4=np.mean(pow_mag60n[226:237,fbin,4])
m70n0=np.mean(pow_mag70n[228:239,fbin,0])
m70n1=np.mean(pow_mag70n[228:239,fbin,1])
m70n4=np.mean(pow_mag70n[228:239,fbin,4])
m80n0=np.mean(pow_mag80n[242:253,fbin,0])
m80n1=np.mean(pow_mag80n[242:253,fbin,1])
m80n4=np.mean(pow_mag80n[242:253,fbin,4])
m90n0=np.mean(pow_mag90n[214:225,fbin,0])
m90n1=np.mean(pow_mag90n[214:225,fbin,1])
m90n4=np.mean(pow_mag90n[214:225,fbin,4])
m100n0=np.mean(pow_mag100n[210:221,fbin,0])
m100n1=np.mean(pow_mag100n[210:221,fbin,1])
m100n4=np.mean(pow_mag100n[210:221,fbin,4])
m110n0=np.mean(pow_mag110n[224:235,fbin,0])
m110n1=np.mean(pow_mag110n[224:235,fbin,1])
m110n4=np.mean(pow_mag110n[224:235,fbin,4])
m120n0=np.mean(pow_mag120n[203:214,fbin,0])
m120n1=np.mean(pow_mag120n[203:214,fbin,1])
m120n4=np.mean(pow_mag120n[203:214,fbin,4])
m130n0=np.mean(pow_mag130n[216:227,fbin,0])
m130n1=np.mean(pow_mag130n[216:227,fbin,1])
m130n4=np.mean(pow_mag130n[216:227,fbin,4])
#m140n0=np.mean(pow_mag140n[206:217,fbin,0])
#m140n1=np.mean(pow_mag140n[206:217,fbin,1])
#m140n4=np.mean(pow_mag140n[206:217,fbin,4])


m10p0=np.mean(pow_mag10p[222:233,fbin,0])
m10p1=np.mean(pow_mag10p[222:233,fbin,1])
m10p4=np.mean(pow_mag10p[222:233,fbin,4])
m20p0=np.mean(pow_mag20p[236:247,fbin,0])
m20p1=np.mean(pow_mag20p[236:247,fbin,1])
m20p4=np.mean(pow_mag20p[236:247,fbin,4])
m30p0=np.mean(pow_mag30p[226:237,fbin,0])
m30p1=np.mean(pow_mag30p[226:237,fbin,1])
m30p4=np.mean(pow_mag30p[226:237,fbin,4])
m40p0=np.mean(pow_mag40p[228:239,fbin,0])
m40p1=np.mean(pow_mag40p[228:239,fbin,1])
m40p4=np.mean(pow_mag40p[228:239,fbin,4])
m50p0=np.mean(pow_mag50p[218:229,fbin,0])
m50p1=np.mean(pow_mag50p[218:229,fbin,1])
m50p4=np.mean(pow_mag50p[218:229,fbin,4])
m60p0=np.mean(pow_mag60p[227:238,fbin,0])
m60p1=np.mean(pow_mag60p[227:238,fbin,1])
m60p4=np.mean(pow_mag60p[227:238,fbin,4])
m70p0=np.mean(pow_mag70p[229:240,fbin,0])
m70p1=np.mean(pow_mag70p[229:240,fbin,1])
m70p4=np.mean(pow_mag70p[229:240,fbin,4])
m80p0=np.mean(pow_mag80p[219:230,fbin,0])
m80p1=np.mean(pow_mag80p[219:230,fbin,1])
m80p4=np.mean(pow_mag80p[219:230,fbin,4])
m90p0=np.mean(pow_mag90p[222:233,fbin,0])
m90p1=np.mean(pow_mag90p[222:233,fbin,1])
m90p4=np.mean(pow_mag90p[222:233,fbin,4])
m100p0=np.mean(pow_mag100p[218:229,fbin,0])
m100p1=np.mean(pow_mag100p[218:229,fbin,1])
m100p4=np.mean(pow_mag100p[218:229,fbin,4])
m110p0=np.mean(pow_mag110p[214:225,fbin,0])
m110p1=np.mean(pow_mag110p[214:225,fbin,1])
m110p4=np.mean(pow_mag110p[214:225,fbin,4])
m120p0=np.mean(pow_mag120p[216:227,fbin,0])
m120p1=np.mean(pow_mag120p[216:227,fbin,1])
m120p4=np.mean(pow_mag120p[216:227,fbin,4])
m130p0=np.mean(pow_mag130p[206:217,fbin,0])
m130p1=np.mean(pow_mag130p[206:217,fbin,1])
m130p4=np.mean(pow_mag130p[206:217,fbin,4])
#m140p0=np.mean(pow_mag140p[215:226,fbin,0])
#m140p1=np.mean(pow_mag140p[215:226,fbin,1])
#m140p4=np.mean(pow_mag140p[215:226,fbin,4])


In [ ]:
## Considering single datapoints from ON pulse 

fbin = 900
s00=pow_mag0[225:226,fbin,0].item()
s01=pow_mag0[225:226,fbin,1].item()
s04=pow_mag0[225:226,fbin,4].item()

s10n0=pow_mag10n[211:212,fbin,0].item()
s10n1=pow_mag10n[211:212,fbin,1].item()
s10n4=pow_mag10n[211:212,fbin,4].item()
s20n0=pow_mag20n[225:226,fbin,0].item()
s20n1=pow_mag20n[225:226,fbin,1].item()
s20n4=pow_mag20n[225:226,fbin,4].item()
s30n0=pow_mag30n[221:222,fbin,0].item()
s30n1=pow_mag30n[221:222,fbin,1].item()
s30n4=pow_mag30n[221:222,fbin,4].item()
s40n0=pow_mag40n[223:224,fbin,0].item()
s40n1=pow_mag40n[223:224,fbin,1].item()
s40n4=pow_mag40n[223:224,fbin,4].item()
s50n0=pow_mag50n[208:209,fbin,0].item()
s50n1=pow_mag50n[208:209,fbin,1].item()
s50n4=pow_mag50n[208:209,fbin,4].item()
s60n0=pow_mag60n[227:228,fbin,0].item()
s60n1=pow_mag60n[227:228,fbin,1].item()
s60n4=pow_mag60n[227:228,fbin,4].item()
s70n0=pow_mag70n[230:231,fbin,0].item()
s70n1=pow_mag70n[230:231,fbin,1].item()
s70n4=pow_mag70n[230:231,fbin,4].item()
s80n0=pow_mag80n[243:244,fbin,0].item()
s80n1=pow_mag80n[243:244,fbin,1].item()
s80n4=pow_mag80n[243:244,fbin,4].item()
s90n0=pow_mag90n[216:217,fbin,0].item()
s90n1=pow_mag90n[216:217,fbin,1].item()
s90n4=pow_mag90n[216:217,fbin,4].item()
s100n0=pow_mag100n[211:212,fbin,0].item()
s100n1=pow_mag100n[211:212,fbin,1].item()
s100n4=pow_mag100n[211:212,fbin,4].item()
s110n0=pow_mag110n[225:226,fbin,0].item()
s110n1=pow_mag110n[225:226,fbin,1].item()
s110n4=pow_mag110n[225:226,fbin,4].item()
s120n0=pow_mag120n[205:206,fbin,0].item()
s120n1=pow_mag120n[205:206,fbin,1].item()
s120n4=pow_mag120n[205:206,fbin,4].item()
s130n0=pow_mag130n[217:218,fbin,0].item()
s130n1=pow_mag130n[217:218,fbin,1].item()
s130n4=pow_mag130n[217:218,fbin,4].item()
#m140n0=np.mean(pow_mag140n[206:217,fbin,0])
#m140n1=np.mean(pow_mag140n[206:217,fbin,1])
#m140n4=np.mean(pow_mag140n[206:217,fbin,4])

s10p0=pow_mag10p[224:225,fbin,0].item()
s10p1=pow_mag10p[224:225,fbin,1].item()
s10p4=pow_mag10p[224:225,fbin,4].item()
s20p0=pow_mag20p[238:239,fbin,0].item()
s20p1=pow_mag20p[238:239,fbin,1].item()
s20p4=pow_mag20p[238:239,fbin,4].item()
s30p0=pow_mag30p[227:228,fbin,0].item()
s30p1=pow_mag30p[227:228,fbin,1].item()
s30p4=pow_mag30p[227:228,fbin,4].item()
s40p0=pow_mag40p[230:231,fbin,0].item()
s40p1=pow_mag40p[230:231,fbin,1].item()
s40p4=pow_mag40p[230:231,fbin,4].item()
s50p0=pow_mag50p[220:221,fbin,0].item()
s50p1=pow_mag50p[220:221,fbin,1].item()
s50p4=pow_mag50p[220:221,fbin,4].item()
s60p0=pow_mag60p[229:230,fbin,0].item()
s60p1=pow_mag60p[229:230,fbin,1].item()
s60p4=pow_mag60p[229:230,fbin,4].item()
s70p0=pow_mag70p[231:232,fbin,0].item()
s70p1=pow_mag70p[231:232,fbin,1].item()
s70p4=pow_mag70p[231:232,fbin,4].item()
s80p0=pow_mag80p[221:222,fbin,0].item()
s80p1=pow_mag80p[221:222,fbin,1].item()
s80p4=pow_mag80p[221:222,fbin,4].item()
s90p0=pow_mag90p[224:225,fbin,0].item()
s90p1=pow_mag90p[224:225,fbin,1].item()
s90p4=pow_mag90p[224:225,fbin,4].item()
s100p0=pow_mag100p[221:222,fbin,0].item()
s100p1=pow_mag100p[221:222,fbin,1].item()
s100p4=pow_mag100p[221:222,fbin,4].item()
s110p0=pow_mag110p[215:216,fbin,0].item()
s110p1=pow_mag110p[215:216,fbin,1].item()
s110p4=pow_mag110p[215:216,fbin,4].item()
s120p0=pow_mag120p[218:219,fbin,0].item()
s120p1=pow_mag120p[218:219,fbin,1].item()
s120p4=pow_mag120p[218:219,fbin,4].item()
s130p0=pow_mag130p[208:209,fbin,0].item()
s130p1=pow_mag130p[208:209,fbin,1].item()
s130p4=pow_mag130p[208:209,fbin,4].item()
#m140p0=np.mean(pow_mag140p[215:226,fbin,0])
#m140p1=np.mean(pow_mag140p[215:226,fbin,1])
#m140p4=np.mean(pow_mag140p[215:226,fbin,4])


In [ ]:
## averaging 0.5 sec OFF pulse datapoints -- correlator data 

m04_off=np.mean(pow_mag0[235:246,fbin,4])

m10n4_off=np.mean(pow_mag10n[222:233,fbin,4])
m20n4_off=np.mean(pow_mag20n[236:247,fbin,4])
m30n4_off=np.mean(pow_mag30n[232:243,fbin,4])
m40n4_off=np.mean(pow_mag40n[234:245,fbin,4])
m50n4_off=np.mean(pow_mag50n[218:229,fbin,4])
m60n4_off=np.mean(pow_mag60n[238:249,fbin,4])
m70n4_off=np.mean(pow_mag70n[240:251,fbin,4])
m80n4_off=np.mean(pow_mag80n[254:265,fbin,4])
m90n4_off=np.mean(pow_mag90n[226:237,fbin,4])
m100n4_off=np.mean(pow_mag100n[222:233,fbin,4])
m110n4_off=np.mean(pow_mag110n[236:247,fbin,4])
m120n4_off=np.mean(pow_mag120n[215:226,fbin,4])
m130n4_off=np.mean(pow_mag130n[228:239,fbin,4])

m10p4_off=np.mean(pow_mag10p[234:245,fbin,4])
m20p4_off=np.mean(pow_mag20p[248:259,fbin,4])
m30p4_off=np.mean(pow_mag30p[238:249,fbin,4])
m40p4_off=np.mean(pow_mag40p[240:251,fbin,4])
m50p4_off=np.mean(pow_mag50p[230:241,fbin,4])
m60p4_off=np.mean(pow_mag60p[239:250,fbin,4])
m70p4_off=np.mean(pow_mag70p[241:252,fbin,4])
m80p4_off=np.mean(pow_mag80p[231:242,fbin,4])
m90p4_off=np.mean(pow_mag90p[234:245,fbin,4])
m100p4_off=np.mean(pow_mag100p[218:229,fbin,4])
m110p4_off=np.mean(pow_mag110p[226:237,fbin,4])
m120p4_off=np.mean(pow_mag120p[228:239,fbin,4])
m130p4_off=np.mean(pow_mag130p[218:229,fbin,4])

In [ ]:
## Considering single datapoints from OFF pulse 

s04_off=pow_mag0[235:236,fbin,4].item()

s10n4_off=pow_mag10n[222:223,fbin,4].item()
s20n4_off=pow_mag20n[236:237,fbin,4].item()
s30n4_off=pow_mag30n[232:233,fbin,4].item()
s40n4_off=pow_mag40n[234:235,fbin,4].item()
s50n4_off=pow_mag50n[218:219,fbin,4].item()
s60n4_off=pow_mag60n[238:239,fbin,4].item()
s70n4_off=pow_mag70n[240:241,fbin,4].item()
s80n4_off=pow_mag80n[254:255,fbin,4].item()
s90n4_off=pow_mag90n[226:227,fbin,4].item()
s100n4_off=pow_mag100n[222:223,fbin,4].item()
s110n4_off=pow_mag110n[236:237,fbin,4].item()
s120n4_off=pow_mag120n[215:216,fbin,4].item()
s130n4_off=pow_mag130n[228:229,fbin,4].item()

s10p4_off=pow_mag10p[234:235,fbin,4].item()
s20p4_off=pow_mag20p[248:249,fbin,4].item()
s30p4_off=pow_mag30p[238:239,fbin,4].item()
s40p4_off=pow_mag40p[240:241,fbin,4].item()
s50p4_off=pow_mag50p[230:231,fbin,4].item()
s60p4_off=pow_mag60p[239:240,fbin,4].item()
s70p4_off=pow_mag70p[241:242,fbin,4].item()
s80p4_off=pow_mag80p[231:232,fbin,4].item()
s90p4_off=pow_mag90p[234:235,fbin,4].item()
s100p4_off=pow_mag100p[218:219,fbin,4].item()
s110p4_off=pow_mag110p[226:227,fbin,4].item()
s120p4_off=pow_mag120p[228:229,fbin,4].item()
s130p4_off=pow_mag130p[218:219,fbin,4].item()

In [ ]:
## Calculating SD for 0.5sec ON pulse datapoints -- correlator data 


std01=np.std(pow_mag0[223:234,fbin,1])

std10n1=np.std(pow_mag10n[210:221,fbin,1])
std20n1=np.std(pow_mag20n[224:235,fbin,1])
std30n1=np.std(pow_mag30n[220:231,fbin,1])
std40n1=np.std(pow_mag40n[222:233,fbin,1])
std50n1=np.std(pow_mag50n[206:217,fbin,1])
std60n1=np.std(pow_mag60n[226:237,fbin,1])
std70n1=np.std(pow_mag70n[228:239,fbin,1])
std80n1=np.std(pow_mag80n[242:253,fbin,1])
std90n1=np.std(pow_mag90n[214:225,fbin,1])
std100n1=np.std(pow_mag100n[210:221,fbin,1])
std110n1=np.std(pow_mag110n[224:235,fbin,1])
std120n1=np.std(pow_mag120n[203:214,fbin,1])
std130n1=np.std(pow_mag130n[216:227,fbin,1])

std10p1=np.std(pow_mag10p[222:233,fbin,1])
std20p1=np.std(pow_mag20p[236:247,fbin,1])
std30p1=np.std(pow_mag30p[226:237,fbin,1])
std40p1=np.std(pow_mag40p[228:239,fbin,1])
std50p1=np.std(pow_mag50p[218:229,fbin,1])
std60p1=np.std(pow_mag60p[227:238,fbin,1])
std70p1=np.std(pow_mag70p[229:240,fbin,1])
std80p1=np.std(pow_mag80p[219:230,fbin,1])
std90p1=np.std(pow_mag90p[222:233,fbin,1])
std100p1=np.std(pow_mag100p[218:229,fbin,1])
std110p1=np.std(pow_mag110p[214:225,fbin,1])
std120p1=np.std(pow_mag120p[216:227,fbin,1])
std130p1=np.std(pow_mag130p[206:217,fbin,1])

In [ ]:
# autos and crosses means and standard deviations lists-- correlator data 
# autos_0x0 -- <VV*>ref,ON
# crosses_0x1 -- <VtVref*>ON
# autos_1x1_ -- <VV*>t,ON 
# autos_1x1_off -- <VV*>t,OFF
# <VV*>ref,OFF --> 0 (this can be autos_0x0_off)

angles = np.arange(-130, 140, 10)

autos_0x0 = [m130n0, m120n0, m110n0, m100n0, m90n0, m80n0, m70n0, m60n0, m50n0, m40n0, m30n0, m20n0, m10n0, m00, m10p0, m20p0, m30p0, m40p0, m50p0, m60p0, m70p0, m80p0, m90p0, m100p0, m110p0, m120p0, m130p0]
crosses_0x1 = [m130n1, m120n1, m110n1, m100n1, m90n1, m80n1, m70n1, m60n1, m50n1, m40n1, m30n1, m20n1, m10n1, m01, m10p1, m20p1, m30p1, m40p1, m50p1, m60p1, m70p1, m80p1, m90p1, m100p1, m110p1, m120p1, m130p1]
autos_1x1_ = [m130n4, m120n4, m110n4, m100n4, m90n4, m80n4, m70n4, m60n4, m50n4, m40n4, m30n4, m20n4, m10n4, m04, m10p4, m20p4, m30p4, m40p4, m50p4, m60p4, m70p4, m80p4, m90p4, m100p4, m110p4, m120p4, m130p4]
autos_1x1_off = [m130n4_off, m120n4_off, m110n4_off, m100n4_off, m90n4_off, m80n4_off, m70n4_off, m60n4_off, m50n4_off, m40n4_off, m30n4_off, m20n4_off, m10n4_off, m04_off, m10p4_off, m20p4_off, m30p4_off, m40p4_off, m50p4_off, m60p4_off, m70p4_off, m80p4_off, m90p4_off, m100p4_off, m110p4_off, m120p4_off, m130p4_off]
stddev = [std130n1, std120n1, std110n1, std100n1, std90n1, std80n1, std70n1, std60n1, std50n1, std40n1, std30n1, std20n1, std10n1, std01, std10p1, std20p1, std30p1, std40p1, std50p1, std60p1, std70p1, std80p1, std90p1, std100p1, std110p1, std120p1, std130p1]


In [ ]:
# Single point corr power lists --

angles = np.arange(-130, 140, 10)

spt_autos_0x0 = [s130n0, s120n0, s110n0, s100n0, s90n0, s80n0, s70n0, s60n0, s50n0, s40n0, s30n0, s20n0, s10n0, s00, s10p0, s20p0, s30p0, s40p0, s50p0, s60p0, s70p0, s80p0, s90p0, s100p0, s110p0, s120p0, s130p0]
spt_crosses_0x1 = [s130n1, s120n1, s110n1, s100n1, s90n1, s80n1, s70n1, s60n1, s50n1, s40n1, s30n1, s20n1, s10n1,s01, s10p1, s20p1, s30p1, s40p1, s50p1, s60p1, s70p1, s80p1, s90p1, s100p1, s110p1, s120p1, s130p1]
spt_autos_1x1_ = [s130n4, s120n4, s110n4, s100n4, s90n4, s80n4, s70n4, s60n4, s50n4, s40n4, s30n4, s20n4, s10n4, s04, s10p4, s20p4, s30p4, s40p4, s50p4, s60p4, s70p4, s80p4, s90p4, s100p4, s110p4, s120p4, s130p4]
spt_autos_1x1_off = [s130n4_off, s120n4_off, s110n4_off, s100n4_off, s90n4_off, s80n4_off, s70n4_off, s60n4_off, s50n4_off, s40n4_off, s30n4_off, s20n4_off, s10n4_off, s04_off, s10p4_off, s20p4_off, s30p4_off, s40p4_off, s50p4_off, s60p4_off, s70p4_off, s80p4_off, s90p4_off, s100p4_off, s110p4_off, s120p4_off, s130p4_off]
stddev = [std130n1, std120n1, std110n1, std100n1, std90n1, std80n1, std70n1, std60n1, std50n1, std40n1, std30n1, std20n1, std10n1, std01, std10p1, std20p1, std30p1, std40p1, std50p1, std60p1, std70p1, std80p1, std90p1, std100p1, std110p1, std120p1, std130p1]


In [ ]:
## autos1x1 --  <VV*>t,ON - <VV*>t,OFF / <VV*>ref,ON - <VV*>ref,OFF
autos_1x1 = []
[autos_1x1.append((autos_1x1_[i] - autos_1x1_off[i]) / autos_0x0[i]) for i in np.arange(0, np.size(autos_1x1_))]

    
## Calculating 1-sigma error bars on crosses
yerr_low = []
yerr_high = []
[yerr_low.append(crosses_0x1[i] - (1*stddev[i])) for i in np.arange(0,np.size(stddev))]
[yerr_high.append(crosses_0x1[i] + (1*stddev[i])) for i in np.arange(0,np.size(stddev))]
    
## Calculating 2-sigma error bars on crosses
yerr_low2 = []
yerr_high2 = []
[yerr_low2.append(crosses_0x1[i] - (2*stddev[i])) for i in np.arange(0,np.size(stddev))]
[yerr_high2.append(crosses_0x1[i] + (2*stddev[i])) for i in np.arange(0,np.size(stddev))]


In [ ]:
## for single data point -- calculating autos(1,1) as
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
## autos1x1 --  <VV*>t,ON - <VV*>t,OFF / <VV*>ref,ON - <VV*>ref,OFF

spt_autos_1x1 = []
for i in np.arange(0, np.size(spt_autos_1x1_)):
    a1x1 = (spt_autos_1x1_[i] - spt_autos_1x1_off[i]) / spt_autos_0x0[i]
    spt_autos_1x1.append(a1x1) 
    
## calculating 1-sigma error bar on crosses 
spt_yerr_low = []
spt_yerr_high = []
for i in np.arange(0,np.size(stddev)):
    spt_yerr_l=spt_crosses_0x1[i] - (1*stddev[i])
    spt_yerr_low.append(spt_yerr_l)
    spt_yerr_h=spt_crosses_0x1[i] + (1*stddev[i])
    spt_yerr_high.append(spt_yerr_h)
    
## Calculating 2-sigma error bars on crosses
spt_yerr_low2 = []
spt_yerr_high2 = []
for i in np.arange(0,np.size(stddev)):
    spt_yerr_l2=spt_crosses_0x1[i] - (2*stddev[i])
    spt_yerr_low2.append(spt_yerr_l2)
    spt_yerr_h2=spt_crosses_0x1[i] + (2*stddev[i])
    spt_yerr_high2.append(spt_yerr_h2)

In [ ]:
##conversion to log scale  -- correlator data 
# dividing factor -- np.sqrt(autos_0x0)

autolog_0x0=10*np.log10(autos_0x0)- 62.7 # giving offset to match the range of VNA plots 
crosslog_0x1=20*np.log10(crosses_0x1/np.sqrt(autos_0x0)) - 62.5 # giving offset to match the range of VNA plots 
autolog_1x1=10*np.log10(autos_1x1) - 14.75 # giving offset to match the range of VNA plots
autolog_1x1_off=10*np.log10(autos_1x1_off) - 72.75
# above "OFF" offset includes difference between auto on and auto off and the one wrt vna which matches with "auto-ON"
# giving offset to match the range of VNA plots 

## error bars in log scale -- something wrong with log conversions 
yerr_lo=20*np.log10(yerr_low/np.sqrt(autos_0x0)) - 62.5
yerr_hi=20*np.log10(yerr_high/np.sqrt(autos_0x0)) - 62.5
yerr_lo2=20*np.log10(yerr_low2/np.sqrt(autos_0x0)) - 62.5
yerr_hi2=20*np.log10(yerr_high2/np.sqrt(autos_0x0)) - 62.5

In [ ]:
## single data point -- conversion to log scale  -- correlator data 
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
## dividing factor -- np.sqrt(autos_0x0)

spt_autolog_0x0=10*np.log10(spt_autos_0x0)- 62.7 # giving offset to match the range of VNA plots 
spt_crosslog_0x1=20*np.log10(spt_crosses_0x1/np.sqrt(spt_autos_0x0)) - 62.5 # giving offset to match the range of VNA plots 
spt_autolog_1x1=10*np.log10(spt_autos_1x1) - 14.75 # giving offset to match the range of VNA plots
spt_autolog_1x1_off=10*np.log10(spt_autos_1x1_off) - 72.75
# above "OFF" offset includes difference between auto on and auto off and the one wrt vna which matches with "auto-ON"
# giving offset to match the range of VNA plots 

In [ ]:
## Approach 1 with error bars --  in log scale (looks right)
spt_yerr_lo=20*np.log10(spt_yerr_low/np.sqrt(spt_autos_0x0)) - 62.5
spt_yerr_hi=20*np.log10(spt_yerr_high/np.sqrt(spt_autos_0x0)) - 62.5
spt_yerr_lo2=20*np.log10(spt_yerr_low2/np.sqrt(spt_autos_0x0)) - 62.5
spt_yerr_hi2=20*np.log10(spt_yerr_high2/np.sqrt(spt_autos_0x0)) - 62.5

In [ ]:
## Approach 2 with error bars -- at lower angles where mean values are comparable to error bars log value 
# would just increase and at higher angles where mean values are much higher that error bars log value would shrink

##### WRONG APPROACH #####

spt_yerr_lo = 20*np.log10(spt_crosses_0x1/np.sqrt(spt_autos_0x0)) - 20*np.log10(stddev/np.sqrt(spt_autos_0x0)) - 62.5
spt_yerr_hi = 20*np.log10(spt_crosses_0x1/np.sqrt(spt_autos_0x0)) + 20*np.log10(stddev/np.sqrt(spt_autos_0x0)) - 62.5

spt_yerr_lo2 = 20*np.log10(spt_crosses_0x1/np.sqrt(spt_autos_0x0)) - 40*np.log10(stddev/np.sqrt(spt_autos_0x0)) - 62.5
spt_yerr_hi2 = 20*np.log10(spt_crosses_0x1/np.sqrt(spt_autos_0x0)) + 40*np.log10(stddev/np.sqrt(spt_autos_0x0)) - 62.5

In [ ]:
## Aprroach 3 with error bars -- at lower angles where mean values are comparable to error bars log value 
# would just increase and at higher angles where mean values are much higher that error bars log value would shrink

##### WRONG APPROACH #####

spt_yerr_lo = 10*np.log10(spt_crosses_0x1) - 20*np.log10(spt_yerr_low)-15.5
spt_yerr_hi = 10*np.log10(spt_yerr_high) - 20*np.log10(spt_crosses_0x1)-15.5

spt_yerr_lo2 = 10*np.log10(spt_crosses_0x1) - 20*np.log10(spt_yerr_low2)-15.5
spt_yerr_hi2 = 10*np.log10(spt_yerr_high2) - 20*np.log10(spt_crosses_0x1)-15.5



In [ ]:
### Using touchstone files from VNA (.s2p files) ###

In [ ]:
## reading all s2p files and saving them in dictionary "dataframes" first with keys and 
## extracting every file in list "s12f" using key

directory = os.getcwd()
data_folder = os.path.join(directory, "/Volumes/EXTRSSD/7_1_AnechoicChamber/7_1_Anechoic_Chamber_SecondTest/s2p_files/")
s2p_files = os.listdir(data_folder)

prefix_ext=len('.s2p')
sorteds2p_files=sorted(s2p_files, key = lambda x: float(x[:-prefix_ext])) # sorting files from -135 to +135 deg
dataframes = {}

s2pf = []
for file in sorteds2p_files:
    dataframe_name = file.split(".")[0]  # Get the name of the file without the extension
    dataframes[dataframe_name] = Network(os.path.join(data_folder, file))
    s2pf.append(dataframes[dataframe_name]) 
    

In [ ]:
## extracting s12 parameter for frequency bin of a choice using plotting functions of Network class from skrf

s12mag_arr = []
s12mags = []
for i in np.arange(1, len(s2pf), 2, dtype=int):
    s12=s2pf[i]["466.4711632453568MHz"].s12.s_db
    s12mag_arr.append(s12)

for j in np.arange(0, np.floor(len(s2pf)/2), dtype=int):
    s12mags.append(s12mag_arr[j][0][0][0])

In [ ]:
##plotting beam pattern with correlator as well as vna data 

colorsarr=cm.gnuplot2(np.linspace(0,1,1024))
fig = plt.figure(figsize=[10,8])
ax = plt.subplot(111)

# logarithmic scale --
angles1=np.arange(-130,140,10)
angles = np.arange(-130,140,10)
ax.plot(angles1, list(reversed(s12mags)), ".", c=colorsarr[fbin-300], label='VNA measurements (S12)')
ax.plot(angles, autolog_0x0, "x", c=colorsarr[fbin-300], label='Corr: (0,0)')
ax.plot(angles, crosslog_0x1, "s",  c=colorsarr[fbin-200],label='Corr: (0,1)')
ax.plot(angles, autolog_1x1, "^", c=colorsarr[fbin-500],label='Corr: (1,1)')
ax.plot(angles, autolog_1x1_off, ".--", c=colorsarr[fbin], label='noise floor')

ax.fill_between(angles, yerr_lo, yerr_hi, alpha=0.25, color='tab:blue', label='1$\sigma$ error')
ax.fill_between(angles, yerr_lo2, yerr_hi2, alpha=0.15, color='tab:pink', label='2$\sigma$ error')
#ax.legend()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
freq=400+((1024-fbin)*0.390625)
plt.title("Beam map at freq bin {} i.e. {}MHZ".format(fbin, freq))
plt.xlabel("angle (deg)")
plt.ylabel("corr power (dB)")

plt.show()

In [ ]:
## single point -- correlator data 
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
## plotting beam pattern with correlator as well as vna data 

colorsarr=cm.gnuplot2(np.linspace(0,1,1024))
fig = plt.figure(figsize=[10,8])
ax = plt.subplot(111)

# logarithmic scale --
angles1=np.arange(-130,140,10)
angles = np.arange(-130,140,10)
ax.plot(angles1, list(reversed(s12mags)), ".", c=colorsarr[fbin-300], label='VNA measurements (S12)')
ax.plot(angles, spt_autolog_0x0, "x", c=colorsarr[fbin-300], label='Corr: (0,0)')
ax.plot(angles, spt_crosslog_0x1, "s",  c=colorsarr[fbin-200],label='Corr: (0,1)')
ax.plot(angles, spt_autolog_1x1, "^", c=colorsarr[fbin-500],label='Corr: (1,1)')
ax.plot(angles, spt_autolog_1x1_off, ".--", c=colorsarr[fbin], label='noise floor')
#plt.ylim(-20,-10)

ax.fill_between(angles, spt_yerr_lo, spt_yerr_hi, alpha=0.25, color='tab:blue', label='1$\sigma$ error')
ax.fill_between(angles, spt_yerr_lo2, spt_yerr_hi2, alpha=0.15, color='tab:pink', label='2$\sigma$ error')
#ax.legend()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
freq=400+((1024-fbin)*0.390625)
plt.title("Beam map at freq bin {} i.e. {}MHZ".format(fbin, freq))
plt.xlabel("angle (deg)")
plt.ylabel("corr power (dB)")

plt.show()

In [ ]:
## log scale using in-built function scale --- beam mapping 

colorsarr=cm.gnuplot2(np.linspace(0,1,1024))
fig = plt.figure(figsize=[10,8])
ax = plt.subplot(111)

# logarithmic scale --
angles1=np.arange(-130,140,10)
angles = np.arange(-130,140,10)
#ax.plot(angles1, list(reversed(s12mags)), ".", c=colorsarr[fbin-300], label='VNA measurements (S12)')
#ax.plot(angles, spt_autos_0x0, "x", c=colorsarr[fbin-300], label='Corr: (0,0)')
ax.plot(angles, spt_crosses_0x1, "s",  c=colorsarr[fbin-200],label='Corr: (0,1)')
#ax.plot(angles, spt_autos_1x1, "^", c=colorsarr[fbin-500],label='Corr: (1,1)')
ax.plot(angles, spt_autos_1x1_off, ".--", c=colorsarr[fbin], label='noise floor')
#plt.ylim(-20,-10)
plt.yscale("log")


ax.fill_between(angles, spt_yerr_low, spt_yerr_high, alpha=0.25, color='tab:blue', label='1$\sigma$ error')
ax.fill_between(angles, spt_yerr_low2, spt_yerr_high2, alpha=0.15, color='tab:pink', label='2$\sigma$ error')
#ax.legend()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
freq=400+((1024-fbin)*0.390625)
plt.title("Beam map at freq bin {} i.e. {}MHZ".format(fbin, freq))
plt.xlabel("angle (deg)")
plt.ylabel("corr power (dB)")


plt.show()

In [ ]:
## VNA data -- Morgan's code  

In [ ]:
import csv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wfdb
from PIL import Image
from IPython.display import display, Image

In [ ]:
## Morgan's code -- VNA data 
## Get the data files and store them in dataframes where they can be called on as dataframes["name##"]

# Get the current directory
directory = os.getcwd()

# Path to the data folder
data_folder = os.path.join(directory, "/Users/kalyanibhopi94/Downloads/anechoic_chamber_data2/") #replace with your path to the data

# List all files in the data folder
files = os.listdir(data_folder)

# Filter files that start with "antwo" and end with ".csv"
outside_files = [file for file in files if file.startswith("antwo") and file.endswith(".csv")]

# Create a dictionary to hold the dataframes
dataframes = {}

# Read each CSV file into a dataframe and store it in the dictionary
for file in outside_files:
    dataframe_name = file.split(".")[0]  # Get the name of the file without the extension
    dataframes[dataframe_name] = pd.read_csv(os.path.join(data_folder, file))

In [ ]:
## Morgan's code -- VNA data
def csv_reframe(csv_file):
    # Load the CSV file into a DataFrame (assuming csv_file is already a DataFrame)
    df = csv_file
    
    # Step 1: Find the index where the first column contains the value "Frequency [Hz]" and "Bridge Name"
    index_frequency = df.index[df.iloc[:, 0] == "Frequency [Hz]"].tolist()[0]
    index_bridge = df.index[df.iloc[:, 0] == "Bridge Mode"].tolist()

    # Extract suffix_name_1 and suffix_name_2 as strings
    suffix_name_1 = str(df.iloc[index_bridge[0], 1])
    suffix_name_2 = str(df.iloc[index_bridge[0], 5])
    
    # Step 3: Extract the row at index `index_frequency` as new column names
    new_column_names = df.iloc[index_frequency].astype(str).tolist()  # Ensure all names are strings

    # Step 4: Check for duplicate column names and add distinguishing suffix
    seen = {}
    suffix_counter = {}
    
    for i, name in enumerate(new_column_names):
        if name not in seen:
            seen[name] = 1  # First occurrence, use suffix_name_1
            new_column_names[i] += f"_{suffix_name_1}"
        else:
            seen[name] += 1
            if seen[name] == 2:
                new_column_names[i] += f"_{suffix_name_2}"  # Second occurrence, use suffix_name_2
            elif seen[name] % 4 == 3:  # Third occurrence (and every fourth occurrence)
                new_column_names[i] += f"_{suffix_name_1}_1"  # Use suffix_name_1 + _1
            else:
                new_column_names[i] += f"_{suffix_name_2}"  # Fourth occurrence, use suffix_name_2

    # Step 5: Copy and store rows including and after the index_frequency row
    new_df = df.iloc[index_frequency+1:].copy()

    # Step 6: Set the new column names
    new_df.columns = new_column_names

    # Step 7: Remove columns with NaN names
    new_df = new_df.dropna(axis=1, how='all')

    # Return the new DataFrame
    return new_df

In [ ]:
## Morgan's code -- VNA data 
#reframing all the data

# Dictionary to hold reframed dataframes
reframed_dfs = {}

for filename, dataframe in dataframes.items():
    # Extract the number from the filename
    num_str = filename.split("antwo")[1].split(".csv")[0]
    num = int(num_str) if num_str else 0
    # Call csv_reframe function on the current dataframe
    reframed_dfs[f"antwo_{num}"] = csv_reframe(dataframe)

In [ ]:
## Morgan's code -- VNA data 
#Displaying as an example

reframed_dfs["antwo_5"]

In [ ]:
## Morgan's code -- VNA data 
# Define function to retrieve specific CSV files from the dataframes dictionary

def retrieve_files(dataframes, file_numbers):
    retrieved_dataframes = {}
    for num in file_numbers:
        filename = f"antwo_{num}"
        if filename in dataframes:
            retrieved_dataframes[filename] = dataframes[filename]
        else:
            print(f"DataFrame {filename} not found.")
    return retrieved_dataframes

In [ ]:
## Morgan's code -- VNA data 
#place where I define & retrieve the files of the different groups of measurements I want

s21_12 = [5,7,8,11,12,15,16,19,20,23,24,27,28,31,32,35,36,39,40,42,45,46,49,50,53,54,57,58,61,62,65,66,69,70,73,74,77]
s11_22 = [4,6,9,10,13,14,17,18,21,22,25,26,29,30,33,34,37,38,41,43,44,47,48,51,52,55,56,59,60,63,64,67,68,71,72,75,76]

s21_12_data = retrieve_files(reframed_dfs,s21_12)
s11_22_data = retrieve_files(reframed_dfs,s11_22)

In [ ]:
## Morgan's code -- VNA data 
#To Match Kalyani's bins

def interpolate_value(step, total_steps, start_value, end_value):
    # Calculate the fraction of the total range
    fraction = step / total_steps
    
    # Interpolate to find the value at the specified step
    interpolated_value = start_value + fraction * (end_value - start_value)
    
    return interpolated_value

# Given values
total_steps = 1024
start_value = 800e6
end_value = 400e6

step_to_find = 778 #enter whatever you want here

# Calculate the interpolated value at step 245
value_at_step = interpolate_value(step_to_find, total_steps, start_value, end_value)

print(f"The value at bin "+str(step_to_find)+" is: " +str(value_at_step)+" Hz")


In [ ]:
colorsarr=cm.gnuplot2(np.linspace(0,1,1024))
desired_freq = value_at_step  
frequency_column = (s21_12_data['antwo_5']['Frequency [Hz]_S21']).astype(float)

# Find the index of the closest value
fav_freq_index = (np.abs(frequency_column - desired_freq)).idxmin()

# Get the actual closest value
closest_value = frequency_column[fav_freq_index]

#print("Index of Closest Value:", fav_freq_index)
#print("Closest Value:", closest_value)

s21mags = []
s21phases = []
for key, df in s21_12_data.items():
    # Extract magnitude and phase values for S21 from specific row and column indices
    mag = df.iloc[:, 1]
    all_mags_corrected1 = mag.astype(float) 
    mag = all_mags_corrected1[fav_freq_index]
    phase = float(df.iloc[fav_freq_index, 2]) % 360  # Ensure the value is numeric before applying modulo
    if phase > 180:
        phase -= 360
    # Append the values to the arrays
    s21mags.append(mag)
    s21phases.append(phase)

s12mags = []
s12phases = []
for key, df in s21_12_data.items():
    # Extract magnitude and phase values for S12 from specific row and column indices
    mag = df.iloc[:, 4]
    all_mags_corrected1 = mag.astype(float)
    mag = all_mags_corrected1[fav_freq_index]
    phase = float(df.iloc[fav_freq_index, 5]) % 360  # Ensure the value is numeric before applying modulo
    if phase > 180:
        phase -= 360
    # Append the values to the arrays
    # Append the values to the arrays
    s12mags.append(mag)
    s12phases.append(phase)

# Convert angles1 and s21mags to numpy arrays for easier handling

angles1 = np.array([-90,-85,-80,-75,-70,-65,-60,-55,-50,-45,-40,-35,-30,-25,-20,-15,-10,-5,0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90])
mags21_float = np.array([float(value) for value in s21mags])
mags12_float = np.array([float(value) for value in s12mags])

fig = plt.figure(figsize=[7,5])
ax = plt.subplot(111)
# Plotting scatter plot with bolder dots
#plt.scatter(angles1, mags1_float, marker='o', s=20, label='Bicolog Transmission on Chime Antenna', alpha=0.75, edgecolors='black',c='orange')  # Increased 's' for larger dots, 'alpha' for transparency, 'edgecolors' for bolder dots

# Plotting line connecting the points
ax.plot(angles1, mags12_float, "s--", color= colorsarr[fbin], label='VNA (S12) measurements')  # Adjust linestyle, linewidth, and alpha as needed
ax.plot(angles, crosslog_0x1, "v--",  c=colorsarr[fbin-300],label='Corr: (0,1)')
ax.plot(angles, autolog_1x1, "^--", c=colorsarr[fbin-600],label='Corr: (1,1)')
#Marking the actual peaks
#plt.axvline(x=angles1[mags1_float.index(max(mags1_float))], color='pink', linestyle='--')
plt.xlabel('Angle (degrees)')
plt.ylabel('Received power (dB)')
# Adding values to x-axis ticks
#plt.xticks(ticks=all_angles, labels=[f'{angle}' for angle in angles1])
# Adding dB unit to y-axis ticks
#plt.yticks(ticks=plt.yticks()[0], labels=[f'{gain} dB' for gain in plt.yticks()[0]])
plt.title('Freq ' +str(s21_12_data['antwo_5']['Frequency [Hz]_S21'][fav_freq_index])+' MHz (freq bin '+str(step_to_find)+')')
plt.legend(loc='upper left',bbox_to_anchor=(1.05, 1))
plt.grid(True, alpha=0.5)
plt.show()

In [ ]:

### Using touchstone files from VNA (.s2p files) ### -- learning how to use functions -- IGNORE THIS SECTION


In [ ]:
## reading single s2p file 
s00 = Network('/Volumes/EXTRSSD/7_1_AnechoicChamber/7_1_Anechoic_Chamber_SecondTest/s2p_files/000.s2p')

In [ ]:
s00

In [ ]:
s00.plot_s_db() # plot magnitude of s-parameters in log scale

In [ ]:
s00.plot_s_db(0,1) ## S12 parameter

In [ ]:
s00.plot_s_deg() # plot phase of s-parameters in degrees

In [ ]:
s00.plot_s_deg(0,0) # plot phase of s11 

In [ ]:
s00.plot_s_smith() # plot complex s-parameters on Smith Chart

In [ ]:
s00.plot_s_smith(1,1)  # plot complex s22 on smith chart

In [ ]:
np.shape(s00.s) # size of every s2p file

In [ ]:
## slicing the data point you want from s2p file 
s12=s00.s[fbin,0,1]